### Name_normalization code

In [ ]:
txt = _items[0]["json"]["message"]["text"].strip().lower()

if not txt:
    return [{"json": {"search_term": "", "search_type": "", "confidence": "low", "error": "empty_input"}}]

# Zip code extraction
words = txt.replace(',', ' ').replace('.', ' ').replace('?', ' ').replace('!', ' ').split()
for word in words:
    clean_word = word.strip('-')
    if clean_word.isdigit() and len(clean_word) == 5:
        return [{"json": {"search_term": clean_word, "search_type": "zip", "confidence": "high", "error": None}}]
    if '-' in word:
        parts = word.split('-')
        if len(parts) == 2 and parts[0].isdigit() and len(parts[0]) == 5:
            return [{"json": {"search_term": parts[0], "search_type": "zip", "confidence": "high", "error": None}}]

state_abbrevs = {
    'al': 'alabama', 'ak': 'alaska', 'az': 'arizona', 'ar': 'arkansas', 'ca': 'california',
    'co': 'colorado', 'ct': 'connecticut', 'de': 'delaware', 'fl': 'florida', 'ga': 'georgia',
    'hi': 'hawaii', 'id': 'idaho', 'il': 'illinois', 'in': 'indiana', 'ia': 'iowa',
    'ks': 'kansas', 'ky': 'kentucky', 'la': 'louisiana', 'me': 'maine', 'md': 'maryland',
    'ma': 'massachusetts', 'mi': 'michigan', 'mn': 'minnesota', 'ms': 'mississippi', 'mo': 'missouri',
    'mt': 'montana', 'ne': 'nebraska', 'nv': 'nevada', 'nh': 'new hampshire', 'nj': 'new jersey',
    'nm': 'new mexico', 'ny': 'new york', 'nc': 'north carolina', 'nd': 'north dakota', 'oh': 'ohio',
    'ok': 'oklahoma', 'or': 'oregon', 'pa': 'pennsylvania', 'ri': 'rhode island', 'sc': 'south carolina',
    'sd': 'south dakota', 'tn': 'tennessee', 'tx': 'texas', 'ut': 'utah', 'vt': 'vermont',
    'va': 'virginia', 'wa': 'washington', 'wv': 'west virginia', 'wi': 'wisconsin', 'wy': 'wyoming',
    'dc': 'washington dc'
}

city_abv = {
    'nyc': 'new york city', 'la': 'los angeles', 'sf': 'san francisco', 'philly': 'philadelphia',
    'vegas': 'las vegas', 'dc': 'washington dc', 'chi': 'chicago', 'atl': 'atlanta',
    'bk': 'brooklyn', 'bklyn': 'brooklyn', 'qns': 'queens', 'bx': 'bronx', 'si': 'staten island',
    'soho': 'manhattan', 'tribeca': 'manhattan', 'harlem': 'manhattan', 'midtown': 'manhattan',
    'fidi': 'manhattan', 'uws': 'manhattan', 'ues': 'manhattan', 'les': 'manhattan',
    'dumbo': 'brooklyn', 'williamsburg': 'brooklyn', 'bushwick': 'brooklyn',
    'astoria': 'queens', 'flushing': 'queens', 'jamaica': 'queens',
    'jersey city': 'jersey city', 'hoboken': 'hoboken'
}

noise_words = [
    'i', 'to', 'about', 'me', 'tell', 'show', 'give',
    'whats', "what's", 'what', 'how', 'much', 'is', 'are', 'the', 'for', 'in', 'at', 'near', 'around',
    'average', 'avg', 'rent', 'cost', 'price', 'pricing', 'rate', 'rates',
    'apartment', 'apartments', 'studio', 'bedroom', 'br', 'bed', 'living', 'housing',
    'cheap', 'affordable', 'expensive', 'nice', 'good', 'best',
    'area', 'place', 'places', 'neighborhood', 'neighbourhoods', 'neighbourhood', 'side',
    'looking', 'want', 'need', 'find', 'search', 'searching', 'know', 'like', 'information','info'
]

for char in ['?', '!', '.', ',', "'", '"']:
    txt = txt.replace(char, ' ')

cleaned = ' '.join([w for w in txt.split() if w not in noise_words]).strip()

for alias, real_name in city_abv.items():
    if alias in cleaned.split():
        return [{"json": {"search_term": real_name.title(), "search_type": "city", "confidence": "high", "error": None}}]

if cleaned in state_abbrevs:
    return [{"json": {"search_term": state_abbrevs[cleaned].title(), "search_type": "state", "confidence": "high", "error": None}}]

multi_word_cities = [
    'new york', 'los angeles', 'san francisco', 'san diego', 'san jose', 'las vegas',
    'new orleans', 'salt lake city', 'kansas city', 'oklahoma city', 'jersey city',
    'virginia beach', 'long beach', 'santa monica', 'palm springs', 'fort worth',
    'el paso', 'corpus christi', 'st louis', 'st paul', 'des moines', 'baton rouge',
    'grand rapids', 'little rock', 'ann arbor', 'palo alto', 'santa barbara'
]

for city in multi_word_cities:
    if city in cleaned:
        return [{"json": {"search_term": city.title(), "search_type": "city", "confidence": "high", "error": None}}]

words_list = cleaned.split()
if len(words_list) >= 2 and words_list[-1] in state_abbrevs:
    return [{"json": {"search_term": ' '.join(words_list[:-1]).title(), "search_type": "city", "confidence": "high", "error": None}}]

if cleaned:
    return [{"json": {"search_term": cleaned.title(), "search_type": "city", "confidence": "medium", "error": None}}]

fallback = ''.join(c for c in txt if c.isalpha() or c == ' ').strip()
if fallback:
    return [{"json": {"search_term": fallback.title(), "search_type": "city", "confidence": "low", "error": None}}]

return [{"json": {"search_term": "", "search_type": "", "confidence": "low", "error": "no_location_found"}}]

### Avg_rent based on Usr_inp

In [ ]:
tot_rent = 0
c = 0
city_name = ""
county = ""
metro = ""
zips = []
search_term = "this location"

# Check if data is recived 
if not _items or len(_items) == 0:
    return [{"json": {
        "error": "no_data",
        "message": f"Sorry, no data found for {search_term}."
    }}]

for item in _items:
    data = item["json"]
    
    if not data:
        continue
    
    # Capture city, county, metro from first valid row
    if not city_name or city_name == "Unknown":
        city_name = data.get("City", "")
        county = data.get("CountyName", "")
        metro = data.get("Metro", "")
    
    zip_code = data.get("RegionName", "")
    rent = data.get("2025-12-31", "")
    
    if rent and rent != "":
        try:
            tot_rent += float(rent)
            c += 1
            zips.append(str(zip_code))
        except ValueError:
            continue

# No valid rent data found
if c == 0:
    return [{"json": {
        "error": "no_rent_data",
        "message": f"No rent data available for {city_name if city_name else 'this location'}."
    }}]

avg_rent = round(tot_rent / c, 4)

return [{"json": {
    "error": None,
    "city": city_name,
    "county": county,
    "metro": metro,
    "avg_rent": avg_rent,
    "zip_count": c,
    "zips_included": zips
}}]

### Filter_sorting_time

In [ ]:
items = _items
# Filter out empty rows
valid = [i for i in items if i["json"].get("last_city") and i["json"].get("last_rent")]
if not valid:
    return [{"json": {"error": True, "message": "No data found"}}]
# Sort by timestamp descending (newest first)
valid.sort(key=lambda x: x["json"].get("timestamp", ""), reverse=True)
# Return only the newest
return [valid[0]]

### Telegram_message_gen

In [ ]:
## Note: This hardcoded version was used for Iteration 1 due to hardware limitations.
## Iteration 2 uses direct expression in Telegram node instead of Python code.
## See README.md for the expression used.


item = _items[0]["json"]
city = item.get("last_city", "")
rent_str = item.get("last_rent", "")

# Handle empty rent
if not rent_str or rent_str == "":
    return [{
        "json": {
            "error": True,
            "message": "No previous city data found. Please search for a city first."
        }
    }]

rent = float(rent_str)
us_national_avg = 1700
diff_percent = ((rent - us_national_avg) / us_national_avg) * 100

if diff_percent < -15:
    verdict = "UNDERVALUED"
    suggest = "Good time to move — below market rate"
elif diff_percent < -10:
    verdict = "AFFORDABLE"
    suggest = "Decent deal — slightly below average"
elif diff_percent <= 10:
    verdict = "FAIRLY PRICED"
    suggest = "Market rate — negotiate if possible"
elif diff_percent <= 15:
    verdict = "EXPENSIVE"
    suggest = "Consider waiting or look at nearby cities"
else:
    verdict = "OVERPRICED"
    suggest = "Wait for correction or find alternatives"

if rent > 2500:
    alternatives = [
        {"city": "Phoenix", "state": "AZ", "rent": 1450},
        {"city": "Austin", "state": "TX", "rent": 1650},
        {"city": "Denver", "state": "CO", "rent": 1850}
    ]
elif rent > 1800:
    alternatives = [
        {"city": "Columbus", "state": "OH", "rent": 1200},
        {"city": "Indianapolis", "state": "IN", "rent": 1150},
        {"city": "Kansas City", "state": "MO", "rent": 1250}
    ]
else:
    alternatives = [
        {"city": "Memphis", "state": "TN", "rent": 1050},
        {"city": "Oklahoma City", "state": "OK", "rent": 1000},
        {"city": "Tulsa", "state": "OK", "rent": 950}
    ]

return [{
    "json": {
        "city": city,
        "rent": rent,
        "us_national_avg": us_national_avg,
        "diff_percent": round(diff_percent, 1),
        "verdict": verdict,
        "Suggestion": suggest,
        "alternatives": alternatives
    }
}]